In [1]:
import os
import torch
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
sys.path.insert(0, './src')
%matplotlib inline


from metrics import NDCG_binary_at_k_batch, Recall_at_k_batch
from models import MultiVAE, MultiDAE, Multi_our_VAE, MultiHoffmanVAE, Multi_ourHoffman_VAE
from training import train_model
from data import Dataset
from args import get_args
import numpy as np
import pandas as pd

import pdb

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import ncvis
import re
import gc

device = 'cuda:1'

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [ ]:
args = dotdict({})
args.train_batch_size = 500
args.val_batch_size = 10
args.n_val_samples = 100
args.device = device

args.learning_rate = 1e-4
args.n_epoches = 200
args.l2_coeff = 0.01 / args.train_batch_size
args.print_info_ = 1

args.annealing = True

if args.annealing:
    args.total_anneal_steps = 200000
    args.anneal_cap = 0.2
else:
    args.total_anneal_steps = 0
    args.anneal_cap = 1.

device_zero = torch.tensor(0., device=device, dtype=torch.float32)
device_one = torch.tensor(1., device=device, dtype=torch.float32)
std_normal = torch.distributions.Normal(loc=device_zero,
                                    scale=device_one)

In [ ]:
models = [model for model in sorted(os.listdir('./models')) if model.startswith('best') ]
for m in models:
    print('\'' + m + '\',' )

In [ ]:
# models = [
# 'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_5_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiHoffmanVAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True_lrdec_3e-05_lrenc_0.001.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_1_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_2_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_3_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_ourHoffman_VAE_K_5_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_1_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_2_N_2_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_3_N_3_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_1_learnreverse_True_anneal_True_lrdec_3e-05_lrenc_0.001.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_True_anneal_False_lrdec_0.001_lrenc_None.pt',
# 'best_model_Multi_our_VAE_K_5_N_5_learnreverse_True_anneal_True_lrdec_0.001_lrenc_None.pt',
# ]

In [ ]:
def load_model(path, device):
    device_zero = torch.tensor(0., device=device, dtype=torch.float32)
    device_one = torch.tensor(1., device=device, dtype=torch.float32)
    std_normal = torch.distributions.Normal(loc=device_zero,
                                        scale=device_one)
    uniform = torch.distributions.Uniform(low=device_zero, high=device_one)
    torch_log_2 = torch.tensor(np.log(2), device=device, dtype=torch.float32)
    
    model = torch.load(path, map_location=device)
    
    model.eval()
    model.std_normal = std_normal
    if hasattr(model, 'torch_log_2'):
        model.torch_log_2 = torch_log_2
    if hasattr(model, 'transitions'):
        model.transitions = model.transitions.to(device)
        if type(model.transitions) == type(torch.nn.ModuleList([])):
            for i in range(len(model.transitions)):
                model.transitions[i].device = device
                model.transitions[i].device_zero = device_zero
                model.transitions[i].device_one = device_one
        else:
            model.transitions.device = device
            model.transitions.device_zero = device_zero
            model.transitions.device_one = device_one
    if hasattr(model, 'reverse_kernel'):
        model.reverse_kernel = model.reverse_kernel.to(device)
        model.reverse_kernel.device = device
        model.device_one = device_one
        
    return model

In [ ]:
resulting_dict = {'model': [], 'data': [], 'K': [], 'N': [], 'anneal': [], 'learnreverse': [], 'learntransitions': [], 'initstepsize': [],  'data': [], 'recall_5': [], 'recall_10': [], 'recall_20': [], 'recall_50': [], 'recall_100': [], 'ndcg_5': [], 'ndcg_10': [], 'ndcg_20': [], 'ndcg_50': [], 'ndcg_100': []}

In [7]:
# args.data = 'ml20m'
# dataset = Dataset(args, data_dir='./data/')

In [8]:
# './logs/metrics_ml20m_MultiVAE_K_None_N_None_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.txt',
# './logs/metrics_ml20m_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001_learntransitions_True_initstepsize_0.1.txt',
# './logs/metrics_ml20m_Multi_ourHoffman_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001_learntransitions_False_initstepsize_0.005.txt',
# './logs/metrics_ml20m_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001_learntransitions_True_initstepsize_0.1.txt',
# './logs/metrics_ml20m_Multi_our_VAE_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001_learntransitions_False_initstepsize_0.005.txt',

In [9]:
models_list = [
#     [
        './models/best_model_MultiVAE_data_ml20m_K_None_N_None_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None_learntransitions_True_initstepsize_0.1.pt',
    
    
#         './models/best_model_MultiVAE_data_ml20m_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None_learntransitions_True_initstepsize_0.1.pt',
#         './models/best_model_Multi_our_VAE_data_ml20m_K_2_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_False_initstepsize_0.01.pt',
#         './models/best_model_Multi_our_VAE_data_ml20m_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_False_initstepsize_0.01.pt',
#     ],
    
#     [
#         './models/best_model_MultiVAE_data_foursquare_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None_learntransitions_True_initstepsize_0.1.pt',
#         './models/best_model_Multi_our_VAE_data_foursquare_K_2_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_False_initstepsize_0.01.pt',
#         './models/best_model_Multi_our_VAE_data_foursquare_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_False_initstepsize_0.01.pt',
#     ],
    

#     [
#         './models/best_model_MultiVAE_data_gowalla_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None_learntransitions_True_initstepsize_0.01.pt',
#         './models/best_model_Multi_our_VAE_data_gowalla_K_2_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_False_initstepsize_0.01.pt',
#         './models/best_model_Multi_our_VAE_data_gowalla_K_2_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_True_initstepsize_0.01.pt',
#         './models/best_model_Multi_our_VAE_data_gowalla_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_True_initstepsize_0.01.pt',
#         './models/best_model_Multi_our_VAE_data_gowalla_K_3_N_1_learnreverse_False_anneal_False_lrdec_0.001_lrenc_0.001_learntransitions_False_initstepsize_0.01.pt',
#     ],
]

In [10]:
pattern_1 = re.compile(r'best_model_(\w*)_data_([\w\d]*)_K_([\d\w]*)_N_([\d\w]*)_learnreverse_(\w*)_anneal_(\w*)_lrdec_([.\d]*)_lrenc_([.\d\w]*)_learntransitions_([\w]*)_initstepsize_([.\d]*)\.pt')

for i, data in enumerate(['gowalla']):#['ml100k', 'ml20m', 'ml25m']):
    args.data = data
    dataset = Dataset(args, data_dir='./data/')
    models = models_list[i]
    for m in tqdm(models):
        vals = re.findall(pattern_1, m)[0]
        resulting_dict['model'].append(vals[0])
        resulting_dict['data'].append(args.data)
        resulting_dict['K'].append(vals[2])
        resulting_dict['N'].append(vals[3])
        resulting_dict['learnreverse'].append(vals[4])
        resulting_dict['anneal'].append(vals[5])
        resulting_dict['learntransitions'].append(vals[8])
        resulting_dict['initstepsize'].append(vals[9])
        
        model = load_model(m, device)
        ndcg_5 = []
        ndcg_10 = []
        ndcg_20 = []
        ndcg_50 = []
        ndcg_100 = []
        recall_5 = []
        recall_10 = []
        recall_20 = []
        recall_50 = []
        recall_100 = []
        for bnum, batch_val in enumerate(dataset.next_val_batch()):
            reshaped_batch = batch_val[0].repeat((args.n_val_samples, 1))
            out = model(reshaped_batch)[0]  ### Here we are using noise when estimate metrics
            with torch.no_grad():
                pred_val = out.detach().view((args.n_val_samples, *batch_val[0].shape)).mean(0)
                X = batch_val[0].cpu().detach().numpy()
                pred_val = pred_val.cpu().detach().numpy()
                pred_val[X.nonzero()] = -np.inf
                ndcg_5.append(NDCG_binary_at_k_batch(pred_val, batch_val[1], k=5))
                ndcg_10.append(NDCG_binary_at_k_batch(pred_val, batch_val[1], k=10))
                ndcg_20.append(NDCG_binary_at_k_batch(pred_val, batch_val[1], k=20))
                ndcg_50.append(NDCG_binary_at_k_batch(pred_val, batch_val[1], k=50))
                ndcg_100.append(NDCG_binary_at_k_batch(pred_val, batch_val[1], k=100))
                recall_5.append(Recall_at_k_batch(pred_val, batch_val[1], k=5))
                recall_10.append(Recall_at_k_batch(pred_val, batch_val[1], k=10))
                recall_20.append(Recall_at_k_batch(pred_val, batch_val[1], k=20))
                recall_50.append(Recall_at_k_batch(pred_val, batch_val[1], k=50))
                recall_100.append(Recall_at_k_batch(pred_val, batch_val[1], k=100))
            del out
        ndcg_5_dist = np.concatenate(ndcg_5)
        ndcg_10_dist = np.concatenate(ndcg_10)
        ndcg_20_dist = np.concatenate(ndcg_20)
        ndcg_50_dist = np.concatenate(ndcg_50)
        ndcg_100_dist = np.concatenate(ndcg_100)

        recall_5_dist = np.concatenate(recall_5)
        recall_10_dist = np.concatenate(recall_10)
        recall_20_dist = np.concatenate(recall_20)
        recall_50_dist = np.concatenate(recall_50)
        recall_100_dist = np.concatenate(recall_100)

        current_ndcg_5 = ndcg_5_dist.mean()
        current_ndcg_10 = ndcg_10_dist.mean()
        current_ndcg_20 = ndcg_20_dist.mean()
        current_ndcg_50 = ndcg_50_dist.mean()
        current_ndcg_100 = ndcg_100_dist.mean()

        current_recall_5 = recall_5_dist.mean()
        current_recall_10 = recall_10_dist.mean()
        current_recall_20 = recall_20_dist.mean()
        current_recall_50 = recall_50_dist.mean()
        current_recall_100 = recall_100_dist.mean()
        
        resulting_dict['ndcg_5'].append(current_ndcg_5)
        resulting_dict['ndcg_10'].append(current_ndcg_10)
        resulting_dict['ndcg_20'].append(current_ndcg_20)
        resulting_dict['ndcg_50'].append(current_ndcg_50)
        resulting_dict['ndcg_100'].append(current_ndcg_100)

        resulting_dict['recall_5'].append(current_recall_5)
        resulting_dict['recall_10'].append(current_recall_10)
        resulting_dict['recall_20'].append(current_recall_20)
        resulting_dict['recall_50'].append(current_recall_50)
        resulting_dict['recall_100'].append(current_recall_100)
        del model
        gc.collect()
    del dataset

  0%|          | 0/5 [00:00<?, ?it/s]./src/metrics.py:32: RuntimeWarning: invalid value encountered in true_divide
  a = DCG / IDCG
./src/metrics.py:47: RuntimeWarning: invalid value encountered in true_divide
  recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
 20%|██        | 1/5 [00:05<00:21,  5.44s/it]/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'models.Target' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
100%|██████████| 5/5 [01:32<00:00, 18.56s/it]


In [11]:
pd.set_option('display.max_colwidth', 100)

In [12]:
df = pd.DataFrame(resulting_dict)
# df.to_csv('./eval_results_bests.csv', index=False)
df.round(3)

,model,data,K,N,anneal,learnreverse,learntransitions,initstepsize,recall_5,recall_10,recall_20,recall_50,recall_100,ndcg_5,ndcg_10,ndcg_20,ndcg_50,ndcg_100
0,MultiVAE,gowalla,None,None,False,False,True,0.01,0.085,0.094,0.121,0.183,0.242,0.098,0.100,0.109,0.131,0.150
1,Multi_our_VAE,gowalla,2,1,False,False,False,0.01,0.094,0.102,0.126,0.186,0.248,0.103,0.104,0.112,0.134,0.153
2,Multi_our_VAE,gowalla,2,1,False,True,True,0.01,0.088,0.097,0.123,0.185,0.241,0.098,0.098,0.108,0.130,0.148
3,Multi_our_VAE,gowalla,3,1,False,True,True,0.01,0.089,0.094,0.120,0.182,0.251,0.102,0.101,0.110,0.132,0.153
4,Multi_our_VAE,gowalla,3,1,False,False,False,0.01,0.090,0.098,0.122,0.183,0.241,0.101,0.102,0.110,0.131,0.150


In [13]:
print(df.round(3).drop(['learnreverse', 'N', 'initstepsize',], axis=1).to_latex(index=False))

\begin{tabular}{lllllrrrrrrrrrr}
\toprule
         model &     data &     K & anneal & learntransitions &  recall\_5 &  recall\_10 &  recall\_20 &  recall\_50 &  recall\_100 &  ndcg\_5 &  ndcg\_10 &  ndcg\_20 &  ndcg\_50 &  ndcg\_100 \\
\midrule
      MultiVAE &  gowalla &  None &  False &             True &     0.085 &      0.094 &      0.121 &      0.183 &       0.242 &   0.098 &    0.100 &    0.109 &    0.131 &     0.150 \\
 Multi\_our\_VAE &  gowalla &     2 &  False &            False &     0.094 &      0.102 &      0.126 &      0.186 &       0.248 &   0.103 &    0.104 &    0.112 &    0.134 &     0.153 \\
 Multi\_our\_VAE &  gowalla &     2 &  False &             True &     0.088 &      0.097 &      0.123 &      0.185 &       0.241 &   0.098 &    0.098 &    0.108 &    0.130 &     0.148 \\
 Multi\_our\_VAE &  gowalla &     3 &  False &             True &     0.089 &      0.094 &      0.120 &      0.182 &       0.251 &   0.102 &    0.101 &    0.110 &    0.132 &     0.153 \\
 Multi\_

In [12]:
df = pd.DataFrame(resulting_dict)
# df.to_csv('./eval_results_bests.csv', index=False)
df.round(3)

,model,data,K,N,anneal,learnreverse,learntransitions,initstepsize,recall_5,recall_10,recall_20,recall_50,recall_100,ndcg_5,ndcg_10,ndcg_20,ndcg_50,ndcg_100
0,MultiVAE,ml20m,None,None,True,False,True,0.1,0.315,0.334,0.397,0.536,0.661,0.322,0.320,0.337,0.385,0.429
1,MultiVAE,ml20m,None,None,False,False,True,0.1,0.287,0.303,0.362,0.501,0.627,0.295,0.292,0.308,0.355,0.399
2,Multi_our_VAE,ml20m,2,1,False,False,False,0.01,0.308,0.329,0.388,0.528,0.654,0.314,0.313,0.330,0.378,0.421
3,Multi_our_VAE,ml20m,3,1,False,False,False,0.01,0.305,0.324,0.387,0.525,0.648,0.310,0.309,0.327,0.375,0.418
4,MultiVAE,foursquare,None,None,False,False,True,0.1,0.060,0.066,0.093,0.147,0.205,0.069,0.071,0.084,0.107,0.127
5,Multi_our_VAE,foursquare,2,1,False,False,False,0.01,0.065,0.064,0.095,0.155,0.206,0.075,0.071,0.087,0.112,0.130
6,Multi_our_VAE,foursquare,3,1,False,False,False,0.01,0.066,0.070,0.096,0.152,0.205,0.077,0.076,0.089,0.112,0.131


In [15]:
print(df.round(3).drop(['learnreverse', 'N', 'initstepsize',], axis=1).to_latex(index=False))

\begin{tabular}{lllllrrrrrrrrrr}
\toprule
         model &        data &     K & anneal & learntransitions &  recall\_5 &  recall\_10 &  recall\_20 &  recall\_50 &  recall\_100 &  ndcg\_5 &  ndcg\_10 &  ndcg\_20 &  ndcg\_50 &  ndcg\_100 \\
\midrule
      MultiVAE &       ml20m &  None &   True &             True &     0.315 &      0.334 &      0.397 &      0.536 &       0.661 &   0.322 &    0.320 &    0.337 &    0.385 &     0.429 \\
      MultiVAE &       ml20m &  None &  False &             True &     0.287 &      0.303 &      0.362 &      0.501 &       0.627 &   0.295 &    0.292 &    0.308 &    0.355 &     0.399 \\
 Multi\_our\_VAE &       ml20m &     2 &  False &            False &     0.308 &      0.329 &      0.388 &      0.528 &       0.654 &   0.314 &    0.313 &    0.330 &    0.378 &     0.421 \\
 Multi\_our\_VAE &       ml20m &     3 &  False &            False &     0.305 &      0.324 &      0.387 &      0.525 &       0.648 &   0.310 &    0.309 &    0.327 &    0.375 &     0.41

In [13]:
# df = pd.DataFrame(resulting_dict)
# # df.to_csv('./eval_results_bests.csv', index=False)
# df.round(3)

In [12]:
# df = pd.DataFrame(resulting_dict)
# # df.to_csv('./eval_results_bests.csv', index=False)
# df[df['model']!='Multi_ourHoffman_VAE'][df['model']!='MultiDAE'].round(3)
# # df

In [13]:
# print(df[df['model']!='Multi_ourHoffman_VAE'][df['model']!='MultiDAE'].round(3).drop(['learnreverse', 'N'], axis=1)#.to_latex(index=False))

In [14]:
# df[df.noise == 1.].sort_values('ndcg', ascending=False).head(30)

In [15]:
# df[df.noise == 0.].sort_values('ndcg', ascending=False).head(33)

# Parameters output

In [5]:
model = load_model('./models/best_model_Multi_our_VAE_data_ml20m_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.0003_lrenc_0.001_learntransitions_False_initstepsize_0.005.pt', 'cuda')
for i, transition in enumerate(model.transitions):
    print(i)
    print('autoreg:', torch.sigmoid(transition.alpha_logit))
    print('stepsize', torch.exp(transition.gamma))
    print('-' * 100)

0
autoreg: tensor(0.9000, device='cuda:0')
stepsize tensor(0.0050, device='cuda:0')
----------------------------------------------------------------------------------------------------
1
autoreg: tensor(0.9000, device='cuda:0')
stepsize tensor(0.0050, device='cuda:0')
----------------------------------------------------------------------------------------------------
2
autoreg: tensor(0.9000, device='cuda:0')
stepsize tensor(0.0050, device='cuda:0')
----------------------------------------------------------------------------------------------------


In [16]:
models_list = [
    './models/best_model_MultiVAE_data_ml20m_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt',
    './models/best_model_Multi_our_VAE_data_ml20m_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.0003_lrenc_0.001_learntransitions_False_initstepsize_0.005.pt',
    './models/best_model_Multi_our_VAE_data_ml20m_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.0003_lrenc_0.001_learntransitions_True_initstepsize_0.1.pt',
    './models/best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True_lrdec_0.001_lrenc_None.pt',
    './models/best_model_Multi_our_VAE_data_ml20m_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001_learntransitions_True_initstepsize_0.1.pt',
    './models/best_model_Multi_ourHoffman_VAE_data_ml20m_K_3_N_1_learnreverse_True_anneal_True_lrdec_0.0003_lrenc_0.001_learntransitions_False_initstepsize_0.005.pt'
]

In [21]:
for bnum, batch_train in enumerate(dataset.next_train_batch()):
    l2 = torch.sum(batch_train ** 2, 1)[..., None]
    x_normed = batch_train / torch.sqrt(torch.max(l2, torch.ones_like(l2) * 1e-12))
    x = model.dropout(x_normed)
    break

for model_name in models_list:
    print(model_name)
    model = load_model(model_name, device)
    enc_out = model.encoder(x).detach()
    mu, logvar = enc_out[:, :model.q_dims[-1]], enc_out[:, model.q_dims[-1]:]
    std = torch.exp(0.5 * logvar)
    print(std)
    print('min std', torch.min(std))
    print('-' * 100)

./models/best_model_MultiVAE_data_ml20m_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None.pt
tensor([[1.0022, 0.9958, 0.9977,  ..., 0.6612, 0.9921, 0.9933],
        [1.0005, 1.0043, 1.0120,  ..., 0.7581, 0.9967, 0.9844],
        [0.9957, 1.0051, 1.0160,  ..., 0.4933, 1.0019, 0.9804],
        ...,
        [0.9935, 1.0088, 1.0026,  ..., 0.9374, 1.0044, 1.0035],
        [1.0048, 1.0114, 1.0095,  ..., 0.8276, 0.9971, 0.9945],
        [1.0014, 1.0036, 1.0072,  ..., 0.8943, 0.9962, 0.9930]],
       device='cuda:1')
min std tensor(0.0285, device='cuda:1')
----------------------------------------------------------------------------------------------------
./models/best_model_Multi_our_VAE_data_ml20m_K_3_N_1_learnreverse_True_anneal_False_lrdec_0.0003_lrenc_0.001_learntransitions_False_initstepsize_0.005.pt
tensor([[0.9840, 1.0332, 1.0254,  ..., 1.0187, 1.0236, 0.2076],
        [0.9756, 1.0036, 1.0029,  ..., 1.0025, 1.0062, 0.2216],
        [0.9666, 0.9646, 1.0012,  ..., 0.99

# Parameters tuning

In [4]:
args.metric = NDCG_binary_at_k_batch
args.n_epoches = 50

In [5]:
for lr in [0.01, 0.001, 0.0001]:
    print('lr ', lr)
    model = load_model('./models/best_model_MultiVAE_K_None_N_None_learnreverse_False_anneal_True.pt', 'cuda')
    args.learning_rate = lr

    for p in model.parameters():
        p.requires_grad_(False)
    model.eval()
    fixed_decoder = model.decoder
    del model

    layers = [200, 600, dataset.n_items]

    model = MultiVAE(layers, args=args)
    model.decoder = fixed_decoder
    model.to(device)
    train_model(model, dataset, args)

lr  0.01
Sequential(
  (0): Linear(in_features=20108, out_features=600, bias=True)
  (1): Tanh()
  (2): Linear(in_features=600, out_features=400, bias=True)
)
Sequential(
  (0): Linear(in_features=200, out_features=600, bias=True)
  (1): Tanh()
  (2): Linear(in_features=600, out_features=20108, bias=True)
)


  0%|          | 0/50 [00:00<?, ?it/s]

763.05676
493.5768
484.29013


  2%|▏         | 1/50 [00:10<08:28, 10.38s/it]

Best NDCG: 0.41631043509731697
Current NDCG: 0.41631043509731697
463.54306
482.39844
510.4981


...

Best NDCG: 0.4297082162678262
Current NDCG: 0.4297082162678262
435.4895
418.33298
521.2349


 98%|█████████▊| 49/50 [07:54<00:09,  9.55s/it]

Best NDCG: 0.4297082162678262
Current NDCG: 0.4293932150163262
504.02423
381.1876
517.6016


100%|██████████| 50/50 [08:03<00:00,  9.68s/it]

Best NDCG: 0.4298818419895272
Current NDCG: 0.4298818419895272
